In [1]:
"""
    (Successful) Attempt for the controlled version:
"""
import numpy as np
import qiskit
from qiskit.quantum_info import state_fidelity
from numpy import linalg as LA
import qib
import matplotlib.pyplot as plt
import scipy
import h5py

import sys
sys.path.append("../../src/brickwall_sparse")
from utils_sparse import construct_ising_local_term, reduce_list, X, I2, get_perms
from ansatz_sparse import ansatz_sparse
import rqcopt as oc
from scipy.sparse.linalg import expm_multiply
from qiskit.quantum_info import random_statevector

Lx, Ly = (4, 4)
L = Lx*Ly
t = .125
latt = qib.lattice.IntegerLattice((Lx, Ly), pbc=True)
field = qib.field.Field(qib.field.ParticleType.QUBIT, latt)
J, h, g = (1, 0, 3)

hamil = qib.IsingHamiltonian(field, J, h, g).as_matrix()
eigenvalues, eigenvectors = scipy.sparse.linalg.eigsh(hamil, k=10)
idx = eigenvalues.argsort()
eigenvalues_sort = eigenvalues[idx]
eigenvectors_sort = eigenvectors[:,idx]
ground_state = eigenvectors_sort[:, 0]

X = np.array([[0, 1], [1, 0]])
Z = np.array([[1, 0], [0, -1]])
Y = np.array([[0, -1j], [1j, 0]])
I2 = np.array([[1, 0], [0, 1]])

hloc1 = construct_ising_local_term(J, 0, 0, ndim=2)
hloc2 = g*(np.kron(X, I2)+np.kron(I2, X))/4

V1 = scipy.linalg.expm(-1j*t*hloc1/4)
V2 = scipy.linalg.expm(-1j*t*hloc2/2)
V3 = scipy.linalg.expm(-1j*t*hloc1/2)
V4 = scipy.linalg.expm(-1j*t*hloc2)
YZ = np.kron(Y, Z)

perms_v, perms_h = get_perms(Lx, Ly)
perms = [perms_v]*1  + [perms_h]*1


In [2]:
with h5py.File(f"./results/tfim2d_RQCOPT_SPARSE_103_Lx4Ly4_t{t}_layers2_niter8_rS1_2hloc.hdf5", "r") as f:
    Vlist =  f["Vlist"][:]

In [3]:
"""
    Test operations.
"""

state = random_statevector(2**L).data
print("Trotter error of the optimized point: ", np.linalg.norm(ansatz_sparse(Vlist, L, perms, state) - expm_multiply(
    -1j * t * hamil, state), ord=2) )

print('infidelity: ', 1-(state_fidelity(ansatz_sparse(Vlist, L, perms, state), expm_multiply(
    -1j * t * hamil, state))))

Trotter error of the optimized point:  0.09055222487110999
infidelity:  0.008182683799179302


In [7]:
sys.path.append("../../src/controlled_unitary_optimizer")
sys.path.append("../../src/brickwall_ansatz")
from optimize_3q import optimize_3q 
from utils_3q import make_controlled, random_unitary

Xlists_opt = {}
perms_qc = [[0, 1], [0, 2], [1, 2], [0, 2], [0, 1], [1, 2], [0, 2], [0, 1], [1, 2]]
#perms_qc = [[0, 1], [0, 2]]

for i in [0, 1]:
    cU = make_controlled(Vlist[i])
    f_best, err_best, Glist_best = (0, 2, None)
    for _ in range(10):
        Xlist_start = [random_unitary(4) for i in range(len(perms_qc))]
        Xlist, f_iter, err_iter = optimize_3q(L, cU, Xlist_start, perms_qc, niter=1000)
        if err_iter[-1] < err_best:
            f_best, err_best, Xlist_best = (f_iter[-1], err_iter[-1], Xlist)
    print("Best f: ", f_best)
    print("Best err: ", err_best)
    Xlists_opt[i] = Xlist_best

Best f:  -7.999999999999911
Best err:  2.5605360033163214e-07
Best f:  -7.999999999999767
Best err:  3.1887617186252316e-07


In [17]:
from qiskit import Aer, execute, transpile
from qiskit.circuit.library import CYGate, CZGate, IGate, CXGate
from qiskit.converters import circuit_to_dag
from qiskit.providers.aer.noise import NoiseModel, errors
from qiskit import Aer, execute, transpile
from scipy import sparse as sp


qc = qiskit.QuantumCircuit(L+1)


for i, V in enumerate(Vlist):
    layer = i
    if True:
        Glist = Xlists_opt[i]
        qc_3 = qiskit.QuantumCircuit(3)
        for j, G in enumerate(Glist):
            qc_3.unitary( G, (3-1-perms_qc[j][1], 3-1-perms_qc[j][0]))
    
        for perm in perms[layer]:
            for j in range(L//2):
                qc.append(qc_3.to_gate(), [L-perm[2*j]-1, L-perm[2*j+1]-1, L])



ccU_cxs_0125= []
ccU_errs_0125 = []
for t_ in [2*t, 4*t, 8*t]:
    state = random_statevector(2**L).data
    backend = Aer.get_backend("statevector_simulator")

    qc_ext2 = qiskit.QuantumCircuit(L+multi_anc)
    qc_ext2.initialize(state, [i for i in range(L)])
    qc_ext2.x(L)
    for i in range(int(t_/t)):
        qc_ext2.append(qc.to_gate(), [i for i in range(L+multi_anc)])
    backend = Aer.get_backend("statevector_simulator")
    sv2 = execute(transpile(qc_ext2), backend).result().get_statevector().data

    ket_0 = np.array([1, 0])
    ket_1 = np.array([0, 1])
    ket_0_e = np.eye(1)
    ket_1_e = np.eye(1)
    for i in range(multi_anc):
        ket_0_e = np.kron(ket_0_e, ket_0)
        ket_1_e = np.kron(ket_1_e, ket_1)
    exact_v2 = np.kron(ket_1_e, expm_multiply(-1j * t_ * hamil, state))
    err = 1-np.abs(np.vdot(sv2, exact_v2))**2
    
    noise_model = NoiseModel()
    dag = circuit_to_dag(transpile(qc_ext2, basis_gates=noise_model.basis_gates+['unitary', 'initialize', 'cx']))
    count_ops = dag.count_ops()
    print(count_ops)
    
    ccU_errs_0125.append(err)
    ccU_cxs_0125.append(count_ops['unitary'])
    print("Trace norm: ", err)

{'initialize': 1, 'sx': 2, 'unitary': 576}
Trace norm:  0.018715142545104535
{'initialize': 1, 'sx': 2, 'unitary': 1152}
Trace norm:  0.01897692282814034
{'initialize': 1, 'sx': 2, 'unitary': 2304}
Trace norm:  0.04313764091533878
